# подсчитывают количество продаж с группировкой по некоторому интервалу времени (квартал, год)
# подсчитывается коэффииент вариации (std / mean)
# на основании коэффициент вариации товар относится к одной из групп XYZ
# проводится для прогнозирования необходимости поставок на склад (стабильные продажи, сезонные, неконтролируемые продажи)

In [5]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import warnings
warnings.filterwarnings("ignore")

In [2]:
pd.set_option('display.max_columns', 40)
pd.set_option('display.float_format', '{:.2f}'.format)

In [8]:
df = pd.read_csv("../data_drugs1.csv", encoding='1251')
df.head()

,DR_Dat,DR_Tim,DR_NChk,DR_NDoc,DR_Apt,DR_Kkm,DR_TDoc,DR_TPay,DR_CDrugs,DR_NDrugs,DR_Suppl,DR_Prod,DR_Kol,DR_CZak,DR_CRoz,DR_SDisc,DR_CDisc,DR_BCDisc,DR_TabEmpl,DR_VZak,DR_Pos
0,2022-08-01,08:06:18,1272,13002561,13,22589,Розничная реализация,18,144734,ГАСТАЛ №12 ТАБ. Д/РАСС.,Пульс,TEVA Pharvaceutical Industries Ltd,1.00,196.71,270.00,0.00,NaN,NaN,29,1,1.00
1,2022-08-01,08:38:53,1273,13002561,13,22589,Розничная реализация,15,69661,"ТОБРОПТ 0,3% 5МЛ. №1 ГЛ.КАПЛИ ФЛ./КАП. /РОМФАРМ/",Пульс,РОМФАРМ КОМПАНИ ( ROMPHARM ),1.00,106.21,127.00,6.00,9.00,200010004357.00,29,1,1.00
2,2022-08-01,08:55:38,1274,13002561,13,22589,Розничная реализация,18,190635,ЭЛИКВИС 5МГ. №60 ТАБ. П/П/О /ПФАЙЗЕР/БРИСТОЛ-М...,ГРАНД КАПИТАЛ СМОЛЕНСК ООО ФК,Пфайзер,1.00,2320.99,2563.00,76.00,9.00,200010018491.00,29,1,1.00
3,2022-08-01,09:00:40,1275,13002561,13,22589,Розничная реализация,18,276370,АРБИДОЛ МАКСИМУМ 200МГ. №10 КАПС. /ОТИСИФАРМ/Ф...,Пульс,ОТИСИФАРМ ПАО,1.00,445.39,541.00,0.00,NaN,NaN,29,1,1.00
4,2022-08-01,09:04:05,1276,13002561,13,22589,Розничная реализация,15,2303,"ЭНАМ 2,5МГ. №20 ТАБ. /Д-Р РЕДДИ/",Протек,Д-р Редди с Лабораторис Лтд / Dr.REDDY's,1.00,18.04,22.00,1.00,9.00,200010000734.00,29,1,5.00


In [16]:
xyz = df.groupby(['DR_Dat', 'DR_NDrugs']).agg({'DR_Kol': sum}).reset_index()
xyz

,DR_Dat,DR_NDrugs,DR_Kol
0,2022-08-01,"911-ВЕНОЛГОН ГЕЛЬ Д/НОГ ПРИ ТЯЖЕСТИ,БОЛИ,ОТЕКА...",1.00
1,2022-08-01,L-ТИРОКСИН 100МКГ. №100 ТАБ. /БЕРЛИН ХЕМИ/,1.00
2,2022-08-01,L-ТИРОКСИН 50МКГ. №50 ТАБ. /БЕРЛИН ХЕМИ/,1.00
3,2022-08-01,L-ТИРОКСИН 75МКГ. №100 ТАБ. /БЕРЛИН ХЕМИ/,2.00
4,2022-08-01,NF ВАТА ХИРУРГ. СТЕР. 100Г. /НЬЮФАРМ/,1.00
...,...,...,...
1084,2022-08-05,"ЭВО ПАНТЕНОЛ ПОМАДА ГИГИЕН. 2,8Г. [EVO] /АВАНТА/",1.00
1085,2022-08-05,ЭМОКСИПИН 10МГ/МЛ. 5МЛ. №1 ГЛ.КАПЛИ ФЛ. КРЫШ/КАП.,2.00
1086,2022-08-05,ЭСВИЦИН СР-ВО П/ОБЛЫСЕНИЯ ЛОСЬОН-ТОНИК 250МЛ. ...,1.00
1087,2022-08-05,ЭТОРИАКС 90МГ. №7 ТАБ. П/П/О,1.00


In [24]:
xyz.groupby('DR_NDrugs')[['DR_Kol']].apply(lambda x: np.std(x) / x.mean()).reset_index()

,DR_NDrugs,DR_Kol
0,"911-ВЕНОЛГОН ГЕЛЬ Д/НОГ ПРИ ТЯЖЕСТИ,БОЛИ,ОТЕКА...",0.00
1,911-ГЕЛЬ-БАЛЬЗАМ Д/НОГ КОНСКИЙ КАШТАН 100МЛ. ТУБА,0.00
2,911-ГЕЛЬ-БАЛЬЗАМ Д/СУСТАВОВ РАЗОГР. ОКОПНИК+МУ...,0.00
3,L-ТИРОКСИН 100МКГ. №100 ТАБ. /БЕРЛИН ХЕМИ/,0.00
4,L-ТИРОКСИН 150МКГ. №100 ТАБ. /БЕРЛИН ХЕМИ/,0.00
...,...,...
798,ЭСВИЦИН СР-ВО П/ОБЛЫСЕНИЯ ЛОСЬОН-ТОНИК 250МЛ. ...,0.25
799,ЭСПУМИЗАН 40МГ. №25 КАПС. /БЕРЛИН-ХЕМИ/,0.00
800,ЭТОРИАКС 90МГ. №7 ТАБ. П/П/О,0.00
801,ЮНИДОКС СОЛЮТАБ 100МГ. №10 ТАБ.ДИСПЕРГ. /АСТЕЛ...,0.00


In [33]:
df = pd.read_csv("../data_drugs1.csv", encoding='1251')
xyz = df.groupby(['DR_Dat', 'DR_NDrugs']).agg({'DR_Kol': sum}).reset_index()
xyz = xyz.groupby('DR_NDrugs').agg({'DR_Dat':'count'}).reset_index()
relevant_drugs = xyz.loc[xyz['DR_Dat'] > 1]['DR_NDrugs']
relevant_drugs = list(relevant_drugs)
# relevant_drugs

['L-ТИРОКСИН 50МКГ. №50 ТАБ. /БЕРЛИН ХЕМИ/',
 'L-ТИРОКСИН 75МКГ. №100 ТАБ. /БЕРЛИН ХЕМИ/',
 'АЗАРГА 10МГ/МЛ.+5МГ/МЛ. 5МЛ. №1 ГЛ.КАПЛИ ФЛ./КАП.',
 'АМОКСИЦИЛЛИН 500МГ. №16 КАПС. /ХЕМОФАРМ/',
 'АРБИДОЛ МАКСИМУМ 200МГ. №10 КАПС. /ОТИСИФАРМ/ФАРМСТАНДАРТ/',
 'АСЕПТИКА САЛФЕТКА СПИРТОВАЯ 60Х100 №20',
 'АСКОРБИНКА ЛУНТИК ВИТ.С КЛУБНИКА №10 ТАБ. (30Г.) /ФАРМ-ПРО/ТИГОДА-ФАРМ/',
 'АСКОРБИНКА ЛУНТИК ВИТ.С №10 ТАБ. (30Г.) /ФАРМ-ПРО/ТИГОДА-ФАРМ/',
 'АСКОРБИНОВАЯ К-ТА 25МГ. ВИШНЯ №10 ТАБ. КРУТКА САХ. /АСКОПРОМ/',
 'АСКОРБИНОВАЯ К-ТА 25МГ. ЯБЛОКО №10 ТАБ. КРУТКА САХ. /АСКОПРОМ/',
 'АСПАРКАМ №60 ТАБ. /ФАРМАПОЛ-ВОЛГА/',
 'АСПЕРА СУПЕРЧИСТОТЕЛ 3МЛ. ФЛ.',
 'АТЕНОЛОЛ РЕНЕВАЛ 50МГ. №30 ТАБ. /ОБНОВЛЕНИЕ/',
 'АТОПИК КРЕМ-СТИК УСПОКАИВАЮЩИЙ Д/ДЕТ. 0+ 4,9МЛ. ПЕНАЛ [ATOPIC]',
 'АТОРВАСТАТИН-СЗ 20МГ. №60 ТАБ. П/П/О /СЕВЕРНАЯ ЗВЕЗДА/',
 'АФОБАЗОЛ 10МГ. №60 ТАБ.',
 'АЦЕТИЛСАЛИЦИЛОВАЯ К-ТА 500МГ. №20 ТАБ. /ФАРМСТАНДАРТ/',
 'АЦИКЛОВИР-БЕЛУПО 5% 10Г. №1 КРЕМ Д/НАРУЖ.ПРИМ. ТУБА /БЕЛУПО/',
 'БЕТАГИСТИН-СЗ 16МГ. №60 ТАБ

In [37]:
df = df[df['DR_NDrugs'].isin(relevant_drugs)]
xyz = df.groupby(['DR_Dat', 'DR_NDrugs']).agg({'DR_Kol': sum}).reset_index()
xyz = xyz.groupby('DR_NDrugs')[['DR_Kol']].apply(lambda x: x.std() / x.mean()).reset_index()
xyz

,DR_NDrugs,DR_Kol
0,L-ТИРОКСИН 50МКГ. №50 ТАБ. /БЕРЛИН ХЕМИ/,0.00
1,L-ТИРОКСИН 75МКГ. №100 ТАБ. /БЕРЛИН ХЕМИ/,0.47
2,АЗАРГА 10МГ/МЛ.+5МГ/МЛ. 5МЛ. №1 ГЛ.КАПЛИ ФЛ./КАП.,0.00
3,АМОКСИЦИЛЛИН 500МГ. №16 КАПС. /ХЕМОФАРМ/,0.47
4,АРБИДОЛ МАКСИМУМ 200МГ. №10 КАПС. /ОТИСИФАРМ/Ф...,0.43
...,...,...
177,"ЭВО ПАНТЕНОЛ ПОМАДА ГИГИЕН. 2,8Г. [EVO] /АВАНТА/",0.47
178,ЭЛИКВИС 5МГ. №60 ТАБ. П/П/О /ПФАЙЗЕР/БРИСТОЛ-М...,0.00
179,ЭМОКСИПИН 10МГ/МЛ. 5МЛ. №1 ГЛ.КАПЛИ ФЛ. КРЫШ/КАП.,0.47
180,ЭНАЛАПРИЛ 5МГ. №20 ТАБ. /ХЕМОФАРМ/,0.47


In [38]:
xyz['XYZ'] = np.where(xyz['DR_Kol'] < 0.1, 'X', np.where(xyz['DR_Kol'] < 0.25, 'Y', 'Z'))
xyz

,DR_NDrugs,DR_Kol,XYZ
0,L-ТИРОКСИН 50МКГ. №50 ТАБ. /БЕРЛИН ХЕМИ/,0.00,X
1,L-ТИРОКСИН 75МКГ. №100 ТАБ. /БЕРЛИН ХЕМИ/,0.47,Z
2,АЗАРГА 10МГ/МЛ.+5МГ/МЛ. 5МЛ. №1 ГЛ.КАПЛИ ФЛ./КАП.,0.00,X
3,АМОКСИЦИЛЛИН 500МГ. №16 КАПС. /ХЕМОФАРМ/,0.47,Z
4,АРБИДОЛ МАКСИМУМ 200МГ. №10 КАПС. /ОТИСИФАРМ/Ф...,0.43,Z
...,...,...,...
177,"ЭВО ПАНТЕНОЛ ПОМАДА ГИГИЕН. 2,8Г. [EVO] /АВАНТА/",0.47,Z
178,ЭЛИКВИС 5МГ. №60 ТАБ. П/П/О /ПФАЙЗЕР/БРИСТОЛ-М...,0.00,X
179,ЭМОКСИПИН 10МГ/МЛ. 5МЛ. №1 ГЛ.КАПЛИ ФЛ. КРЫШ/КАП.,0.47,Z
180,ЭНАЛАПРИЛ 5МГ. №20 ТАБ. /ХЕМОФАРМ/,0.47,Z
